In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs


Note: you may need to restart the kernel to use updated packages.


In [ ]:


import os


# Retrieve API key from user data if it’s stored there
os.environ["OPENAI_API_KEY"] = "****"


In [3]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [5]:
from langchain_community.graphs import Neo4jGraph

In [6]:
from typing import Tuple, List, Optional


In [7]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser


In [8]:
from langchain_core.runnables import ConfigurableField

In [9]:

from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [10]:

import os

In [11]:

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [12]:
from langchain_community.vectorstores import Neo4jVector


In [13]:
from langchain_community.graphs import Neo4jGraph


In [ ]:
NEO4J_URI="***"
NEO4J_USERNAME="**"
NEO4J_PASSWORD="**"

In [ ]:
os.environ["OPENAI_API_KEY"] ="**"
os.environ["NEO4J_URI"] ="**"
os.environ["NEO4J_USERNAME"] ="**"
os.environ["NEO4J_PASSWORD"] ="**"

In [17]:
!python -m pip install wikipedia

In [18]:
from langchain_community.graphs import Neo4jGraph

In [20]:
graph = Neo4jGraph()

ValueError: Cannot resolve address 541a8429.databases.neo4j.io:7687

In [80]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth 1").load()

In [81]:

len(raw_documents)

25

In [82]:
raw_documents[:3]


[Document(metadata={'title': 'Elizabeth I', 'summary': 'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary\'s reign, Elizabeth was i

In [83]:

from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [84]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [85]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [86]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [87]:
graph_documents

[GraphDocument(nodes=[Node(id='Elizabeth I', type='Person', properties={}), Node(id='England', type='Country', properties={}), Node(id='Ireland', type='Country', properties={}), Node(id='House Of Tudor', type='Monarchy', properties={}), Node(id='Henry Viii', type='Person', properties={}), Node(id='Anne Boleyn', type='Person', properties={}), Node(id='Edward Vi', type='Person', properties={}), Node(id='Lady Jane Grey', type='Person', properties={}), Node(id='Mary', type='Person', properties={}), Node(id='William Cecil', type='Person', properties={}), Node(id='Baron Burghley', type='Person', properties={}), Node(id='James Vi Of Scotland', type='Person', properties={}), Node(id='Mary, Queen Of Scots', type='Person', properties={}), Node(id='France', type='Country', properties={}), Node(id='Spain', type='Country', properties={}), Node(id='Sir Francis Walsingham', type='Person', properties={})], relationships=[Relationship(source=Node(id='Elizabeth I', type='Person', properties={}), target=

In [88]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [89]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"


In [90]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [76]:

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass


In [77]:

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [79]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [41]:
from typing import Tuple, List, Optional


In [42]:
from langchain_community.vectorstores import Neo4jVector


In [43]:

from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [44]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")



[]

In [46]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [47]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [48]:

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [49]:
entity_chain = prompt | llm.with_structured_output(Entities)


In [50]:
entity_chain.invoke({"question": "Where was Amelia Earhart born?"}).names


['Amelia Earhart']

In [51]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars


In [52]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [53]:
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [54]:
print(structured_retriever("Who is Elizabeth I?"))


Elizabeth I - RULED -> England
Elizabeth I - RULED -> Ireland
Elizabeth I - BELONGS_TO -> House Of Tudor
Elizabeth I - PARENT -> Henry Viii
Elizabeth I - PARENT -> Anne Boleyn
Elizabeth I - SIBLING -> Edward Vi
Elizabeth I - DEPENDS_ON -> William Cecil
Elizabeth I - DEPENDS_ON -> Sir Francis Walsingham
Elizabeth I - CREATED -> Baron Burghley
Elizabeth I - ESTABLISHED_CHURCH -> England
Elizabeth I - SUPREME_GOVERNOR -> England
Elizabeth I - FOREIGN_AFFAIRS -> France
Elizabeth I - FOREIGN_AFFAIRS -> Spain
Edward Vi - IGNORED_CLAIMS -> Elizabeth I
Mary - IMPRISONED -> Elizabeth I
Queen Elizabeth I - INVOLVED_IN -> Portraiture


In [55]:

def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [56]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [57]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


In [58]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [59]:
_search_query = RunnableBranch(
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),

    RunnableLambda(lambda x : x["question"]),
)


In [60]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""


In [61]:
prompt = ChatPromptTemplate.from_template(template)


In [62]:

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [63]:
chain.invoke({"question": "Which house did Elizabeth I belong to?"})


Search query: Which house did Elizabeth I belong to?


'Elizabeth I belonged to the House of Tudor.'

In [64]:
chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)

Search query: When was Elizabeth I born?


'Elizabeth I was born on 7 September 1533.'